In [1]:
import dash

from dash import dcc, html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from folium.plugins import MarkerCluster
import folium
import requests
import json
from joblib import load

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [3]:
# Verbindung zur Datenbank herstellen
try:
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print("Verbindung zur Datenbank erfolgreich hergestellt.")

    # Einen Cursor erstellen, um SQL-Abfragen auszuführen
    cursor = connection.cursor()

    # 1. SQL SELECT-Abfrage für die letzten 4 Einträge je senseid_fk
    select_query = """
        SELECT *
        FROM (
            SELECT *, ROW_NUMBER() OVER (PARTITION BY senseid_fk ORDER BY time DESC) AS r
            FROM value
        ) sub
        WHERE r <= 4;
    """
    cursor.execute(select_query)
    rows = cursor.fetchall()

    # DataFrame für die Ergebnisse der ersten Abfrage erstellen
    df_new = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

    # 2. SQL SELECT-Abfrage für die Wetterstationen
    select_stations_query = ("SELECT ort, lat, lon, senseid FROM box_id;")
    cursor.execute(select_stations_query)
    station = cursor.fetchall()

    # DataFrame für die Ergebnisse der zweiten Abfrage erstellen


    # 3. SQL JOIN-Abfrage für Werte und Stationen
    select_value_query = """
        SELECT v.*, b.ort
        FROM value v
        JOIN box_id b ON v.senseid_fk = b.senseid;
    """
    cursor.execute(select_value_query)
    values = cursor.fetchall()

    # DataFrame für die Ergebnisse der dritten Abfrage erstellen
    df= pd.DataFrame(values, columns=[desc[0] for desc in cursor.description])

    # Cursor und Verbindung schließen
    cursor.close()
    connection.close()

    # Verarbeitung der DataFrames oder Rückgabe
    # Hier kannst du die DataFrames df_last_entries, df_stations und df_combined weiterverarbeiten

except psycopg2.Error as e:
    print(f"Fehler beim Verbinden zur Datenbank: {e}")


Verbindung zur Datenbank erfolgreich hergestellt.


In [4]:
station

[('Mersch', 49.754663, 6.105682, '5ae4a726223bd80019a367a6'),
 ('Ettlingen', 48.926209, 8.374409, '5dde9523ba7944001da4150e'),
 ('Stuttgart', 48.77961, 9.213723, '5c08379b1c28f9001a3f580c'),
 ('München', 48.146546, 11.576033, '5b4d11485dc1ec001b5452c7'),
 ('Berlin', 52.51127, 13.397356, '65e8d93acbf5700007f920ca'),
 ('Hamburg', 53.544046, 9.997499, '590e0b0a51d3460011c725c4'),
 ('Dortmund', 51.49735, 7.494606, '605f498077a88b001bba3dc0'),
 ('Jena', 50.934561, 11.589701, '64722d1c9be0580007f776d9')]

In [5]:
# Extrahiere die einzelnen Komponenten
df_new['jahr'] = df_new['time'].dt.year
df_new['monat'] = df_new['time'].dt.month
df_new['tag'] = df_new['time'].dt.day
df_new['stunde'] = df_new['time'].dt.hour
df_new['minute'] = df_new['time'].dt.minute

In [6]:
modell1 = load("Temperatur_in_10_Minuten_model.joblib")
modell2 = load("Temperatur_in_20_Minuten_model.joblib")
modell3 = load("Temperatur_in_10_Minuten_model.joblib")

In [7]:
target_variables = ['Temperatur_in_10_Minuten', 'Temperatur_in_20_Minuten', 'Temperatur_in_30_Minuten']
models = {'Temperatur_in_10_Minuten':modell1,'Temperatur_in_20_Minuten': modell2, 'Temperatur_in_30_Minuten': modell3}
lags = [1, 2, 3]  # Lags für 10, 20 und 30 Minuten
for lag in lags:
    df_new[f'Temperatur_lag_{lag*10}'] = df_new.groupby('senseid_fk')['temperature'].shift(lag)

# Fehlende Werte durch geeignete Werte ersetzen, falls notwendig
#df_all.fillna(method='ffill', inplace=True)

# Liste der einzigartigen Stationen
stations = df_new['senseid_fk'].unique()

# Dictionary zum Speichern der Vorhersagen für jede Station
all_predictions = {}

for st in stations:
    # Daten für die aktuelle Station auswählen
    df_station = df_new[df_new['senseid_fk'] == st]
    
    # Neueste Zeile für Vorhersagen auswählen
    latest_row = df_station.iloc[[-1]]
    
    # Vorhersagen treffen
    predictions = {}
    for target in target_variables:
        model = models[target]
        # Die Features für das Modell auswählen (identisch zu den Trainingsdaten)
        X_new = latest_row.drop(target_variables + ["time", "index"], axis=1, errors='ignore')
        
        # Vorhersage durchführen
        prediction = model.predict(X_new)
        predictions[target] = prediction[0]
    
    # Ergebnisse für die aktuelle Station speichern
    all_predictions[st] = predictions

# Ergebnisse in ein DataFrame umwandeln
df_predictions = pd.DataFrame(all_predictions)

In [8]:
from dash import State



# Load the data from the API
response = requests.get("https://us-central1-skillful-coast-423214-a6.cloudfunctions.net/lastEntries")
df_latest = pd.read_json(json.dumps(response.json()))

# Rename and clean up DataFrame columns
df_latest.columns = ["index", "time", "temperature", "luftfeuchtigkeit", "pm10", "pm2.5", "senseidfk", "r"]
df_latest.drop(columns="r", inplace=True)

# Connect to the PostgreSQL database to get station info


# Initialize the map centered on Germany
map_deutschland = folium.Map(location=[51.1657, 10.4515], zoom_start=6)

# Create a Marker Cluster
marker_cluster = MarkerCluster().add_to(map_deutschland)

# Add weather stations to the map
for stati in station:
    ort, lat, lon, senseid = stati
    # Filter the DataFrame for the current station
    if senseid in df_predictions.columns:
        prediction_data = df_predictions[senseid].values
    else:
        prediction_data = [None] * 3  # Set None if no prediction data available
    station_data = df_latest[df_latest['senseidfk'] == senseid]
    
    # Create a popup with the station's current data
    if not station_data.empty:
        latest_data = station_data.iloc[-1]
        popup_text = (
            f"Station: {ort}<br>"
            f"Time: {latest_data['time']}<br>"
            f"Temperature: {latest_data['temperature']} °C<br>"
            f"Luftfeuchtigkeit: {latest_data['luftfeuchtigkeit']} %<br>"
            f"PM10: {latest_data['pm10']} µg/m³<br>"
            f"PM2.5: {latest_data['pm2.5']} µg/m³"
            f"Temperatur in 10 Minuten: {prediction_data[0]}<br>"
            f"Temperatur in 20 Minuten: {prediction_data[1]}<br>"
            f"Temperatur in 30 Minuten: {prediction_data[2]}"
        )
    else:
        popup_text = f"Station: {ort}<br>No data available."
    
    folium.Marker(
        location=[lat, lon],
        popup=popup_text
    ).add_to(marker_cluster)

# Display the map
map_deutschland.save('map_deutschland.html')



station_data

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_11384\1671237428.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_latest = pd.read_json(json.dumps(response.json()))


,index,time,temperature,luftfeuchtigkeit,pm10,pm2.5,senseidfk
6,1735879,"Thu, 27 Jun 2024 20:50:03 GMT",24.74,59.68,9.18,3.72,64722d1c9be0580007f776d9


Werte Droppen

In [9]:
df = df[df['luftfeuchtigkeit'] != 100.00]
df = df[df['luftfeuchtigkeit'] != 99.90]
df = df[df["pm10"]<1200]
bedingung = df['temperature'] > df['temperature'].quantile(0.99)
df = df.drop(df[bedingung].index)

In [10]:
stations = df['ort'].unique()
stations_options = [{'label': 'Alle Stationen', 'value': 'all'}] + [{'label': station, 'value': station} for station in stations]

In [14]:
df = df.sort_values("time")

In [ ]:
# Fetch the data from the API

app = dash.Dash(__name__)
app.layout = html.Div(id='app-container', children=[
    html.H1(children='Umweltüberwachung Dashboard', id='title'),
    
    html.Button('Kartenansicht', id='map-view-button', n_clicks=0, style={'margin-right': '10px'}),
    html.Button('Kennzahlenansicht', id='metrics-view-button', n_clicks=0),

        html.Button('Toggle Dark/Light Mode', id='toggle-button', n_clicks=0, style={'margin-top': '20px'}),
    
    html.Div(id='control-container', children=[
        html.Div(
            children='Wählen Sie eine Station und eine Kennzahl aus, um die Daten anzuzeigen.',
            id='description',
        ),

        dcc.Dropdown(
            id='station-dropdown',
            options=stations_options,
            value='all',
        ),

        html.Div(
            children='Wählen Sie eine Kennzahl:',
            id='metric-label',
            style={'marginTop': 20}
        ),
        
        dcc.RadioItems(
            id='metric-radio',
            options=[
                {'label': 'Durchschnitt', 'value': 'mean'},
                {'label': 'Median', 'value': 'median'},
                {'label': 'Varianz', 'value': 'var'},
                {'label': 'Standardabweichung', 'value': 'std'},
                {'label': 'Min', 'value': 'min'},
                {'label': 'Max', 'value': 'max'},
                {'label': 'Boxplot', 'value': 'boxplot'}
            ],
            value='mean',
            labelStyle={'display': 'inline-block', 'marginRight': 10}
        ),

        html.Div(
            children='Wählen Sie die Aggregationsstufe:',
            id='aggregation-label',
            style={'marginTop': 20}
        ),
        
        dcc.Checklist(
            id='aggregation-checklist',
            options=[
                {'label': 'Jahr', 'value': 'year'},
                {'label': 'Monat', 'value': 'month'},
                {'label': 'Tag', 'value': 'day'},
                {'label': 'Stunde', 'value': 'hour'}
            ],
            value=['year', 'month'],
            labelStyle={'display': 'inline-block', 'marginRight': 10}
        ),
        
        html.Div(
            children='Anzeigen der letzten 7 Tage:',
            id='last-7-days-label',
            style={'marginTop': 20}
        ),
        
        dcc.Checklist(
            id='last-7-days-checklist',
            options=[
                {'label': 'Letzte 7 Tage', 'value': 'last_7_days'}
            ],
            value=[],
            labelStyle={'display': 'inline-block', 'marginRight': 10}
        )
    ]),


# Layout der Dash-App definieren

    html.Div(id='view-container', children=[
        dcc.Graph(id='temperature-graph'),
        dcc.Graph(id='luftfeuchtigkeit-graph'),
        dcc.Graph(id='pm-graph'),
    ])

    
])

# Funktion zur Berechnung von IQR
def calculate_metrics(group, metric):
    if metric == 'iqr':
        return group.quantile(0.75) - group.quantile(0.25)
    else:
        return group.agg(metric)
    
@app.callback(
    [Output('view-container', 'children'),
     Output('control-container', 'style')],
    [Input('map-view-button', 'n_clicks'), Input('metrics-view-button', 'n_clicks')]
)
def switch_views(map_view_clicks, metrics_view_clicks):
    if map_view_clicks > metrics_view_clicks:
        return [html.Iframe(srcDoc=open('map_deutschland.html', 'r').read(), width='100%', height='600')], {'display': 'none'}
    else:
        return [
            dcc.Graph(id='temperature-graph'),
            dcc.Graph(id='luftfeuchtigkeit-graph'),
            dcc.Graph(id='pm-graph')
        ], {'display': 'block'}


# Callback zur Aktualisierung der Diagramme
@app.callback(
    [Output('temperature-graph', 'figure'),
     Output('luftfeuchtigkeit-graph', 'figure'),
     Output('pm-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('metric-radio', 'value'),
     Input('aggregation-checklist', 'value'),
     Input('last-7-days-checklist', 'value'),
     Input('toggle-button', 'n_clicks')]
)

def update_graphs(selected_station, selected_metric, selected_aggregation, last_7_days, n_clicks):
    if not selected_station or not selected_metric:
        raise PreventUpdate

    # Filter the DataFrame by the selected station
    if selected_station != 'all':
        filtered_df = df[df['ort'] == selected_station]
    else:
        filtered_df = df.copy()

    # Clean the DataFrame of NaN values
    filtered_df = filtered_df.dropna(subset=['temperature', 'luftfeuchtigkeit', 'pm2_5', 'pm10'])

    # Filter the data for the last 7 days if selected
    if 'last_7_days' in last_7_days:
        last_7_days_date = filtered_df['time'].max() - pd.Timedelta(days=7)
        filtered_df = filtered_df[filtered_df['time'] >= last_7_days_date]

        temp_fig = px.line(filtered_df, x='time', y='temperature', title='Temperatur der letzten 7 Tage')
        humidity_fig = px.line(filtered_df, x='time', y='luftfeuchtigkeit', title='Luftfeuchtigkeit der letzten 7 Tage')
        pm_fig = px.line(filtered_df, x='time', y=['pm2_5', 'pm10'], title='PM2.5 und PM10 der letzten 7 Tage')

    elif selected_metric == 'boxplot':
        # Create Boxplots for the entire DataFrame
        temp_fig = px.box(filtered_df, y='temperature', title='Temperatur Boxplot')
        humidity_fig = px.box(filtered_df, y='luftfeuchtigkeit', title='Luftfeuchtigkeit Boxplot')
        pm_fig = px.box(filtered_df, y=['pm2_5', 'pm10'], title='PM2.5 und PM10 Boxplot')

    else:
        # Ensure valid aggregation combinations only if not 'boxplot'
        if 'hour' in selected_aggregation and 'day' not in selected_aggregation:
            selected_aggregation.append('day')
        if 'day' in selected_aggregation and 'month' not in selected_aggregation:
            selected_aggregation.append('month')
        if 'month' in selected_aggregation and 'year' not in selected_aggregation:
            selected_aggregation.append('year')

        # Create the aggregation column based on the selection
        aggregation = ''
        if 'year' in selected_aggregation:
            aggregation += '%Y'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

        if 'month' in selected_aggregation:
            aggregation += '-%m'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
            filtered_df['aggregation'] = pd.to_datetime(filtered_df['aggregation'], format='%Y-%m')
         

        if 'day' in selected_aggregation:
            aggregation += '-%d'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)

        if 'hour' in selected_aggregation:
            aggregation += ' %H'
            filtered_df['aggregation'] = filtered_df['time'].dt.strftime(aggregation)
        if aggregation == '-%d':
            filtered_df['aggregation'] = filtered_df['time'].dt.day_of_year

        

        # Plot based on the selected metric
        aggregation_mapping = {
            'mean': 'mean',
            'median': 'median',
            'var': 'var',
            'std': 'std',
            'min': 'min',
            'max': 'max'
        }


        df_temp = filtered_df.groupby('aggregation').agg({'temperature': aggregation_mapping[selected_metric]}).reset_index()
        #print(df_temp)
       
        temp_fig = px.line(df_temp
, 
                            x='aggregation', y='temperature', title=f'Temperatur ({selected_metric.capitalize()})')
        temp_fig.update_traces(line=dict(color='red'))
        humidity_fig = px.line(filtered_df.groupby('aggregation').agg({'luftfeuchtigkeit': aggregation_mapping[selected_metric]}).reset_index(), 
                                x='aggregation', y='luftfeuchtigkeit', title=f'Luftfeuchtigkeit ({selected_metric.capitalize()})')
        humidity_fig.update_traces(line=dict(color='blue'))
        pm_fig = px.line(filtered_df.groupby('aggregation').agg({'pm2_5': aggregation_mapping[selected_metric], 'pm10': aggregation_mapping[selected_metric]}).reset_index(), 
                        x='aggregation', y=['pm2_5', 'pm10'], title=f'PM2.5 und PM10 ({selected_metric.capitalize()})')
        pm_fig.for_each_trace(lambda trace: trace.update(line=dict(color='blue')) if trace.name == 'pm2_5' else trace.update(line=dict(color='red')))

    if n_clicks % 2 == 0:
        graph_layout = dict(plot_bgcolor='black', paper_bgcolor='black', font=dict(color='white'))
        temp_fig.update_layout(graph_layout)
        humidity_fig.update_layout(graph_layout)
        pm_fig.update_layout(graph_layout)
    else:
        graph_layout = dict(plot_bgcolor='white', paper_bgcolor='white', font=dict(color='black'))
        temp_fig.update_layout(graph_layout)
        humidity_fig.update_layout(graph_layout)
        pm_fig.update_layout(graph_layout)

    return temp_fig, humidity_fig, pm_fig

# Callback zur Umschaltung des Modus
@app.callback(
    [Output('title', 'style'),
     Output('toggle-button', 'style'),
     Output('description', 'style'),
     Output('metric-label', 'style'),
     Output('aggregation-label', 'style'),
     Output('last-7-days-label', 'style'),
     Output('station-dropdown', 'style'),
     Output('metric-radio', 'labelStyle'),
     Output('aggregation-checklist', 'labelStyle'),
     Output('last-7-days-checklist', 'labelStyle'),
     Output('app-container', 'style')],
    [Input('toggle-button', 'n_clicks')]
)
def toggle_dark_light_mode(n_clicks):
    if n_clicks % 2 == 0:
        dark_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        light_mode_style = {'color': 'white', 'backgroundColor': 'black'}
        dropdown_style = {'backgroundColor': 'black', 'color': 'white'}
        container_style = {'backgroundColor': 'black'}
    else:
        dark_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        light_mode_style = {'color': 'black', 'backgroundColor': 'white'}
        dropdown_style = {'backgroundColor': 'white', 'color': 'black'}
        container_style = {'backgroundColor': 'white'}

    return (dark_mode_style, light_mode_style, dark_mode_style, dark_mode_style,
            dark_mode_style, dark_mode_style, dropdown_style, light_mode_style, 
            light_mode_style, light_mode_style, container_style)

@app.callback(
    [Output('aggregation-checklist', 'options'),
     Output('aggregation-checklist', 'value')],
    [Input('metric-radio', 'value')]
)
def toggle_aggregation_checklist(selected_metric):
    if selected_metric == 'boxplot':
        return [{'label': 'Jahr', 'value': 'year', 'disabled': True},
                {'label': 'Monat', 'value': 'month', 'disabled': True},
                {'label': 'Tag', 'value': 'day', 'disabled': True},
                {'label': 'Stunde', 'value': 'hour', 'disabled': True}], []
    else:
        return [{'label': 'Jahr', 'value': 'year', 'disabled': False},
                {'label': 'Monat', 'value': 'month', 'disabled': False},
                {'label': 'Tag', 'value': 'day', 'disabled': False},
                {'label': 'Stunde', 'value': 'hour', 'disabled': False}], ['year', 'month']
# App ausführen
if __name__ == '__main__':
    app.run_server(debug=True,host='127.0.0.1', port=8050,jupyter_mode = "external")


Dash app running on http://127.0.0.1:8050/
           index                    time  temperature  luftfeuchtigkeit  pm10  \
1117465  1212404 2022-04-07 00:00:36.089         9.48             91.20  3.50   
1125781  1220720 2022-04-07 00:02:49.812        13.70             81.00  1.20   
1117464  1212403 2022-04-07 00:05:37.798         9.44             91.41  3.30   
1125780  1220719 2022-04-07 00:06:14.152        13.70             80.70  1.30   
1125779  1220718 2022-04-07 00:09:37.765        13.60             80.50  1.30   
...          ...                     ...          ...               ...   ...   
1640865  1735866 2024-06-27 20:40:01.000        29.70             48.60  2.30   
1640866  1735867 2024-06-27 20:40:02.000        19.60             70.42  2.33   
1640867  1735868 2024-06-27 20:40:02.000        32.80             39.80  0.00   
1640869  1735870 2024-06-27 20:40:03.000        22.78              0.00  0.00   
1640870  1735871 2024-06-27 20:40:03.000        25.09             

   aggregation  temperature
0   2022-04-01    16.346340
1   2022-05-01    22.513157
2   2022-06-01    25.603114
3   2022-07-01    26.844905
4   2022-08-01    24.923448
5   2022-09-01    16.492528
6   2022-10-01    19.402339
7   2022-11-01     8.636959
8   2022-12-01     5.778318
9   2023-01-01    10.261029
10  2023-02-01     9.679348
11  2023-03-01    12.328768
12  2023-04-01    14.722573
13  2023-05-01    19.185834
14  2023-06-01    24.896838
15  2023-07-01    23.510585
16  2023-08-01    23.414725
17  2023-09-01    21.870545
18  2023-10-01    14.626256
19  2023-11-01     5.948662
20  2023-12-01     8.296007
21  2024-01-01     4.754216
22  2024-02-01     9.776024
23  2024-03-01    11.422756
24  2024-04-01    13.931607
25  2024-05-01    21.341481
26  2024-06-01    21.755828


c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



           index                    time  temperature  luftfeuchtigkeit  pm10  \
1117465  1212404 2022-04-07 00:00:36.089         9.48             91.20  3.50   
1125781  1220720 2022-04-07 00:02:49.812        13.70             81.00  1.20   
1117464  1212403 2022-04-07 00:05:37.798         9.44             91.41  3.30   
1125780  1220719 2022-04-07 00:06:14.152        13.70             80.70  1.30   
1125779  1220718 2022-04-07 00:09:37.765        13.60             80.50  1.30   
...          ...                     ...          ...               ...   ...   
1640865  1735866 2024-06-27 20:40:01.000        29.70             48.60  2.30   
1640866  1735867 2024-06-27 20:40:02.000        19.60             70.42  2.33   
1640867  1735868 2024-06-27 20:40:02.000        32.80             39.80  0.00   
1640869  1735870 2024-06-27 20:40:03.000        22.78              0.00  0.00   
1640870  1735871 2024-06-27 20:40:03.000        25.09             58.10  8.38   

         pm2_5             

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



...

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



           index                    time  temperature  luftfeuchtigkeit  pm10  \
1110674  1205615 2022-04-13 16:01:43.264         21.9              68.9  5.37   
1110673  1205614 2022-04-13 16:04:10.734         21.9              68.8  9.67   
1110672  1205613 2022-04-13 16:06:38.353         21.8              68.7  8.73   
1110671  1205612 2022-04-13 16:09:05.915         21.9              69.0  9.47   
1110670  1205611 2022-04-13 16:11:33.481         21.7              69.0  8.40   
...          ...                     ...          ...               ...   ...   
1640831  1735832 2024-06-27 20:00:00.000         31.7              57.8  4.62   
1640839  1735840 2024-06-27 20:10:00.000         31.4              58.7  4.70   
1640847  1735848 2024-06-27 20:20:01.000         31.1              61.7  8.87   
1640855  1735856 2024-06-27 20:30:01.000         30.9              63.3  5.75   
1640863  1735864 2024-06-27 20:40:00.000         31.0              63.6  5.30   

         pm2_5             

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



           index                    time  temperature  luftfeuchtigkeit  pm10  \
1125781  1220720 2022-04-07 00:02:49.812         13.7              81.0  1.20   
1125780  1220719 2022-04-07 00:06:14.152         13.7              80.7  1.30   
1125779  1220718 2022-04-07 00:09:37.765         13.6              80.5  1.30   
1125778  1220717 2022-04-07 00:13:01.587         13.5              80.6  1.23   
1125777  1220716 2022-04-07 00:16:25.300         13.5              81.1  1.20   
...          ...                     ...          ...               ...   ...   
1640833  1735834 2024-06-27 20:00:01.000         29.9              46.3  3.10   
1640841  1735842 2024-06-27 20:10:01.000         29.6              47.7  2.30   
1640849  1735850 2024-06-27 20:20:01.000         29.6              48.1  2.30   
1640857  1735858 2024-06-27 20:30:02.000         29.8              48.5  2.20   
1640865  1735866 2024-06-27 20:40:01.000         29.7              48.6  2.30   

         pm2_5             

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\Benjamin\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

